# Binance Top 50 Crypto Data collector

This notebook retrieves the top 50 cryptocurrencies according to Coingecko, checks if they are trading on Binance, and if they are collects
- Hourly price candles for 192 hours (8 days)
- Gets current price of each crypto
- Outputs the data in a csv file

In [5]:
import requests
import pandas as pd
from datetime import datetime, timezone
from IPython.display import display

BASE_URL = "https://data-api.binance.vision"
COINGECKO_TOP_50_URL = "https://api.coingecko.com/api/v3/coins/markets"

## Gets the current top 50 Coins from Coingecko

In [6]:
def get_top_50_marketcap():
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False
    }
    response = requests.get(COINGECKO_TOP_50_URL, params=params)
    return response.json()

## Get All Trading Pairs from Binance to cross match Coingecko List
This will be used to make sure that the coin is trading on Binance with USDT.

In [9]:
def get_binance_symbols():
    url = f"{BASE_URL}/api/v3/exchangeInfo?"
    response = requests.get(url)
    data = response.json()
    symbols = []

    for item in data['symbols']:
        symbols.append(item['symbol'])
    return symbols

## Fetch the Crypto's hourly candles for 7 Days.
Each coin's price history is fetched in 1h intervals. We use 192 candles to represent roughly 8 days so that we have buffer data to exclude the current days candles.

In [11]:
def get_price_history(symbol, interval="1h", limit=192):
    url = f"{BASE_URL}/api/v3/klines?"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    data = requests.get(url, params=params).json()
    
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base', 'taker_buy_quote', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df[['close']]

## Get the Current Price of the Cryptocurrency
Uses a api call to get the current price of the cryptocurrency in accordance with the last completed hourly candle.

In [16]:
def get_current_price(symbol):
    url = f"{BASE_URL}/api/v3/ticker/price"
    params = {"symbol": symbol}
    response = requests.get(url, params=params)

    data = response.json()
    # price = float(data['price'])
    # return price
    if 'price' in data:
        return float(data['price'])
    else:
        print(f"Price data not available for {symbol}")
        return None

## Calculate Daily closing prices for the last 7 days
- Refactors the hourly candles into daily by resampling and retrieves the last value of each day

In [7]:
def get_7_day_closes(df):
    daily = df['close'].resample('1D').last()
    # Exclude today's partial data if exists
    if daily.index[-1].date() == pd.Timestamp.utcnow().date():
        daily = daily[:-1]
    return daily.tail(7).tolist()

## Fetch and Store data on top 50 Cryptocurrencies
- This is the main function that will call the helper functions to get price data and store to a csv

In [18]:
def fetch_and_store(output_csv="top50_crypto_info.csv"):
    top_50 = get_top_50_marketcap()
    binance_symbols = get_binance_symbols()
    coin_info = []

    for coin in top_50:
        symbol = coin['symbol'].upper()
        name = coin['name']
        # check binancepair
        binance_pair = symbol + "USDT"
        snapshot_time_utc = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")

        if binance_pair not in binance_symbols:
            print(f"Skipping { coin['symbol'].upper()}  { coin['name']} — not on Binance as a trading pair with USDT")
            continue

        df = get_price_history(binance_pair)
        closing_prices_7d = get_7_day_closes(df)
        current_price = get_current_price(binance_pair)
        # print(coin['symbol'].upper())
        # print(current_price)

        coin_info.append({
            "timestamp_utc": snapshot_time_utc,
            "symbol": symbol,
            "name": name,
            "current_price": round(current_price, 2),
            "price_1d_ago": round(closing_prices_7d[6], 2),
            "price_2d_ago": round(closing_prices_7d[5], 2),
            "price_3d_ago": round(closing_prices_7d[4], 2),
            "price_4d_ago": round(closing_prices_7d[3], 2),
            "price_5d_ago": round(closing_prices_7d[2], 2),
            "price_6d_ago": round(closing_prices_7d[1], 2),
            "price_7d_ago": round(closing_prices_7d[0], 2),
        })  
    df_coin_info = pd.DataFrame(coin_info)
    display(df_coin_info)
    df_coin_info.to_csv(output_csv, index=False)

if __name__ == "__main__":
    fetch_and_store()


Skipping USDT  Tether — not on Binance as a trading pair with USDT
Skipping STETH  Lido Staked Ether — not on Binance as a trading pair with USDT
Skipping HYPE  Hyperliquid — not on Binance as a trading pair with USDT
Skipping WSTETH  Wrapped stETH — not on Binance as a trading pair with USDT
Skipping LEO  LEO Token — not on Binance as a trading pair with USDT
Skipping WETH  WETH — not on Binance as a trading pair with USDT
Skipping WBT  WhiteBIT Coin — not on Binance as a trading pair with USDT
Skipping WEETH  Wrapped eETH — not on Binance as a trading pair with USDT
Skipping BSC-USD  Binance Bridged USDT (BNB Smart Chain) — not on Binance as a trading pair with USDT
Skipping BGB  Bitget Token — not on Binance as a trading pair with USDT
Skipping USDE  Ethena USDe — not on Binance as a trading pair with USDT
Skipping CBBTC  Coinbase Wrapped BTC — not on Binance as a trading pair with USDT
Skipping PI  Pi Network — not on Binance as a trading pair with USDT
Skipping SUSDE  Ethena Stake

,timestamp_utc,symbol,name,current_price,price_1d_ago,price_2d_ago,price_3d_ago,price_4d_ago,price_5d_ago,price_6d_ago,price_7d_ago
0,2025-06-30 12:40 UTC,BTC,Bitcoin,107645.07,108356.93,107296.79,107047.59,106947.06,107340.58,106083.00,105333.93
1,2025-06-30 12:40 UTC,ETH,Ethereum,2463.28,2500.09,2435.62,2423.17,2415.96,2418.49,2448.45,2411.66
2,2025-06-30 12:40 UTC,XRP,XRP,2.18,2.21,2.19,2.14,2.11,2.19,2.19,2.16
3,2025-06-30 12:40 UTC,BNB,BNB,653.86,654.90,648.60,645.71,642.30,646.07,643.53,640.49
4,2025-06-30 12:40 UTC,SOL,Solana,150.23,153.32,150.72,142.09,139.06,143.57,145.92,144.51
5,2025-06-30 12:40 UTC,USDC,USDC,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
6,2025-06-30 12:40 UTC,TRX,TRON,0.28,0.28,0.27,0.27,0.27,0.27,0.27,0.27
7,2025-06-30 12:40 UTC,DOGE,Dogecoin,0.16,0.17,0.16,0.16,0.16,0.16,0.17,0.16
8,2025-06-30 12:40 UTC,ADA,Cardano,0.56,0.58,0.57,0.56,0.55,0.57,0.59,0.58
9,2025-06-30 12:40 UTC,WBTC,Wrapped Bitcoin,107590.50,108282.69,107245.76,106951.08,106866.77,107237.99,106014.61,105305.21
